## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-25-04-24-32 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/24/us/tru...
1,2026-02-25-04-23-44 +0000,nypost,Trump’s record-breaking State of the Union 202...,https://nypost.com/2026/02/24/us-news/trumps-r...
2,2026-02-25-04-22-26 +0000,startribune,"In retort to Trump, Mayor Jacob Frey says all ...",https://www.startribune.com/mayor-jacob-frey-s...
3,2026-02-25-04-21-06 +0000,nypost,Fury as developers use hidden ‘loophole’ to bu...,https://nypost.com/2026/02/24/us-news/the-la-c...
4,2026-02-25-04-10-47 +0000,nypost,Shameless female shoplifter steals nearly $100...,https://nypost.com/2026/02/24/us-news/brazen-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2467/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,65
5,union,32
181,new,32
676,war,20
51,address,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2026-02-25-03-04-21 +0000,nyt,"“After four years, in which millions and milli...",https://www.nytimes.com/live/2026/02/24/us/tru...,197
383,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,170
65,2026-02-25-02-43-12 +0000,nypost,"From USA hockey team to 100-year-old veteran, ...",https://nypost.com/2026/02/24/us-news/usa-chan...,164
72,2026-02-25-02-28-12 +0000,nypost,Rep. Al Green booted from State of the Union 2...,https://nypost.com/2026/02/24/us-news/rep-al-g...,159
10,2026-02-25-04-02-00 +0000,wsj,President Trump delivered a State of the Union...,https://www.wsj.com/politics/policy/five-takea...,154


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,197,2026-02-25-03-04-21 +0000,nyt,"“After four years, in which millions and milli...",https://www.nytimes.com/live/2026/02/24/us/tru...
383,170,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...
68,99,2026-02-25-02-38-52 +0000,nyt,"“As thousands of new businesses are forming, f...",https://www.nytimes.com/live/2026/02/24/us/tru...
7,85,2026-02-25-04-06-50 +0000,nyt,“They’ve killed and maimed thousands of Americ...,https://www.nytimes.com/live/2026/02/24/us/tru...
28,83,2026-02-25-03-35-45 +0000,nyt,"“With the great Big Beautiful Bill, we gave yo...",https://www.nytimes.com/live/2026/02/24/us/tru...
36,71,2026-02-25-03-24-12 +0000,nyt,“Under my just-enacted most favored nation agr...,https://www.nytimes.com/live/2026/02/24/us/tru...
300,67,2026-02-24-15-48-20 +0000,cbc,"Carney pledges $2B for Ukraine, sanctions for ...",https://www.cbc.ca/news/politics/carney-ukrain...
27,64,2026-02-25-03-35-53 +0000,nyt,“I want to stop all payments to big insurance ...,https://www.nytimes.com/live/2026/02/24/us/tru...
117,59,2026-02-24-23-54-43 +0000,nypost,Warner Bros. Discovery may upend Netflix deal ...,https://nypost.com/2026/02/24/media/warner-bro...
273,47,2026-02-24-17-00-00 +0000,nypost,OpenAI asks California watchdog to probe who’s...,https://nypost.com/2026/02/24/business/openai-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
